In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu copy


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_co2(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_co2(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(100)
test_Y = user_function_co2(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(1020)
train_Y = user_function_co2(train_X)

[[ 7.          9.45162131 25.60359496 46.44081944  1.          2.74939229]
 [13.         21.59743161 20.76089687 45.91045986  1.          3.17805255]
 [ 3.         21.70180484  9.82821519 45.07008265  1.          1.51071148]
 [19.         23.44338368 12.59436683 55.2372177   3.          4.17731049]
 [ 3.         10.65862289 17.94829774 43.08557951  3.          3.66968183]
 [ 6.         13.17646686 19.91647441 48.46612031  3.          2.64246776]
 [ 6.         18.28621561 15.44314858 30.42325159  1.          2.07636219]
 [15.         18.99034969 26.37370339 58.12417705  3.          1.7672483 ]
 [ 3.         12.96107914  5.20000897 48.07783975  2.          4.25438391]
 [ 7.         22.07782524 13.00566836 52.67560204  2.          2.53844788]
 [14.         13.68241606  6.11843784 46.09622042  3.          1.78163872]
 [17.         20.26760528 12.08602375 62.37707293  2.          2.36077284]
 [12.         10.76957138  6.67728354 59.30819797  2.          2.61709301]
 [19.         13.87503778

 Retrying in 1 seconds

Evaluating input: 2 of 100

 Retrying in 1 seconds

Evaluating input: 3 of 100

 Retrying in 1 seconds

Evaluating input: 4 of 100

 Retrying in 1 seconds

Evaluating input: 5 of 100

 Retrying in 1 seconds

Evaluating input: 6 of 100

 Retrying in 1 seconds

Evaluating input: 7 of 100

 Retrying in 1 seconds

Evaluating input: 8 of 100

 Retrying in 1 seconds

Evaluating input: 9 of 100

 Retrying in 1 seconds

Evaluating input: 10 of 100

 Retrying in 1 seconds

Evaluating input: 11 of 100

 Retrying in 1 seconds

Evaluating input: 12 of 100

 Retrying in 1 seconds

Evaluating input: 13 of 100

 Retrying in 1 seconds

Evaluating input: 14 of 100

 Retrying in 1 seconds

Evaluating input: 15 of 100

 Retrying in 1 seconds

Evaluating input: 16 of 100

 Retrying in 1 seconds

Evaluating input: 17 of 100

 Retrying in 1 seconds

Evaluating input: 18 of 100

 Retrying in 1 seconds

Evaluating input: 19 of 100

 Retrying in 1 seconds

Evaluating input: 20 of 100

 


Evaluating input: 46 of 1020

 Retrying in 1 seconds

Evaluating input: 47 of 1020

 Retrying in 1 seconds

Evaluating input: 48 of 1020

 Retrying in 1 seconds

Evaluating input: 49 of 1020

 Retrying in 1 seconds

Evaluating input: 50 of 1020

 Retrying in 1 seconds

Evaluating input: 51 of 1020

 Retrying in 1 seconds

Evaluating input: 52 of 1020

 Retrying in 1 seconds

Evaluating input: 53 of 1020

 Retrying in 1 seconds

Evaluating input: 54 of 1020

 Retrying in 1 seconds

Evaluating input: 55 of 1020

 Retrying in 1 seconds

Evaluating input: 56 of 1020

 Retrying in 1 seconds

Evaluating input: 57 of 1020

 Retrying in 1 seconds

Evaluating input: 58 of 1020

 Retrying in 1 seconds

Evaluating input: 59 of 1020

 Retrying in 1 seconds

Evaluating input: 60 of 1020

 Retrying in 1 seconds

Evaluating input: 61 of 1020

 Retrying in 1 seconds

Evaluating input: 62 of 1020

 Retrying in 1 seconds

Evaluating input: 63 of 1020

 Retrying in 1 seconds

Evaluating input: 64 of 102


Evaluating input: 196 of 1020

 Retrying in 1 seconds

Evaluating input: 197 of 1020

 Retrying in 1 seconds

Evaluating input: 198 of 1020

 Retrying in 1 seconds

Evaluating input: 199 of 1020

 Retrying in 1 seconds

Evaluating input: 200 of 1020

 Retrying in 1 seconds

Evaluating input: 201 of 1020

 Retrying in 1 seconds

Evaluating input: 202 of 1020

 Retrying in 1 seconds

Evaluating input: 203 of 1020

 Retrying in 1 seconds

Evaluating input: 204 of 1020

 Retrying in 1 seconds

Evaluating input: 205 of 1020

 Retrying in 1 seconds

Evaluating input: 206 of 1020

 Retrying in 1 seconds

Evaluating input: 207 of 1020

 Retrying in 1 seconds

Evaluating input: 208 of 1020

 Retrying in 1 seconds

Evaluating input: 209 of 1020

 Retrying in 1 seconds

Evaluating input: 210 of 1020

 Retrying in 1 seconds

Evaluating input: 211 of 1020

 Retrying in 1 seconds

Evaluating input: 212 of 1020

 Retrying in 1 seconds

Evaluating input: 213 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 345 of 1020

 Retrying in 1 seconds

Evaluating input: 346 of 1020

 Retrying in 1 seconds

Evaluating input: 347 of 1020

 Retrying in 1 seconds

Evaluating input: 348 of 1020

 Retrying in 1 seconds

Evaluating input: 349 of 1020

 Retrying in 1 seconds

Evaluating input: 350 of 1020

 Retrying in 1 seconds

Evaluating input: 351 of 1020

 Retrying in 1 seconds

Evaluating input: 352 of 1020

 Retrying in 1 seconds

Evaluating input: 353 of 1020

 Retrying in 1 seconds

Evaluating input: 354 of 1020

 Retrying in 1 seconds

Evaluating input: 355 of 1020

 Retrying in 1 seconds

Evaluating input: 356 of 1020

 Retrying in 1 seconds

Evaluating input: 357 of 1020

 Retrying in 1 seconds

Evaluating input: 358 of 1020

 Retrying in 1 seconds

Evaluating input: 359 of 1020

 Retrying in 1 seconds

Evaluating input: 360 of 1020

 Retrying in 1 seconds

Evaluating input: 361 of 1020

 Retrying in 1 seconds

Evaluating input: 362 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 494 of 1020

 Retrying in 1 seconds

Evaluating input: 495 of 1020

 Retrying in 1 seconds

Evaluating input: 496 of 1020

 Retrying in 1 seconds

Evaluating input: 497 of 1020

 Retrying in 1 seconds

Evaluating input: 498 of 1020

 Retrying in 1 seconds

Evaluating input: 499 of 1020

 Retrying in 1 seconds

Evaluating input: 500 of 1020

 Retrying in 1 seconds

Evaluating input: 501 of 1020

 Retrying in 1 seconds

Evaluating input: 502 of 1020

 Retrying in 1 seconds

Evaluating input: 503 of 1020

 Retrying in 1 seconds

Evaluating input: 504 of 1020

 Retrying in 1 seconds

Evaluating input: 505 of 1020

 Retrying in 1 seconds

Evaluating input: 506 of 1020

 Retrying in 1 seconds

Evaluating input: 507 of 1020

 Retrying in 1 seconds

Evaluating input: 508 of 1020

 Retrying in 1 seconds

Evaluating input: 509 of 1020

 Retrying in 1 seconds

Evaluating input: 510 of 1020

 Retrying in 1 seconds

Evaluating input: 511 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 643 of 1020

 Retrying in 1 seconds

Evaluating input: 644 of 1020

 Retrying in 1 seconds

Evaluating input: 645 of 1020

 Retrying in 1 seconds

Evaluating input: 646 of 1020

 Retrying in 1 seconds

Evaluating input: 647 of 1020

 Retrying in 1 seconds

Evaluating input: 648 of 1020

 Retrying in 1 seconds

Evaluating input: 649 of 1020

 Retrying in 1 seconds

Evaluating input: 650 of 1020

 Retrying in 1 seconds

Evaluating input: 651 of 1020

 Retrying in 1 seconds

Evaluating input: 652 of 1020

 Retrying in 1 seconds

Evaluating input: 653 of 1020

 Retrying in 1 seconds

Evaluating input: 654 of 1020

 Retrying in 1 seconds

Evaluating input: 655 of 1020

 Retrying in 1 seconds

Evaluating input: 656 of 1020

 Retrying in 1 seconds

Evaluating input: 657 of 1020

 Retrying in 1 seconds

Evaluating input: 658 of 1020

 Retrying in 1 seconds

Evaluating input: 659 of 1020

 Retrying in 1 seconds

Evaluating input: 660 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 792 of 1020

 Retrying in 1 seconds

Evaluating input: 793 of 1020

 Retrying in 1 seconds

Evaluating input: 794 of 1020

 Retrying in 1 seconds

Evaluating input: 795 of 1020

 Retrying in 1 seconds

Evaluating input: 796 of 1020

 Retrying in 1 seconds

Evaluating input: 797 of 1020

 Retrying in 1 seconds

Evaluating input: 798 of 1020

 Retrying in 1 seconds

Evaluating input: 799 of 1020

 Retrying in 1 seconds

Evaluating input: 800 of 1020

 Retrying in 1 seconds

Evaluating input: 801 of 1020

 Retrying in 1 seconds

Evaluating input: 802 of 1020

 Retrying in 1 seconds

Evaluating input: 803 of 1020

 Retrying in 1 seconds

Evaluating input: 804 of 1020

 Retrying in 1 seconds

Evaluating input: 805 of 1020

 Retrying in 1 seconds

Evaluating input: 806 of 1020

 Retrying in 1 seconds

Evaluating input: 807 of 1020

 Retrying in 1 seconds

Evaluating input: 808 of 1020

 Retrying in 1 seconds

Evaluating input: 809 of 1020

 Retrying in 1 seconds

Evaluatin


Evaluating input: 941 of 1020

 Retrying in 1 seconds

Evaluating input: 942 of 1020

 Retrying in 1 seconds

Evaluating input: 943 of 1020

 Retrying in 1 seconds

Evaluating input: 944 of 1020

 Retrying in 1 seconds

Evaluating input: 945 of 1020

 Retrying in 1 seconds

Evaluating input: 946 of 1020

 Retrying in 1 seconds

Evaluating input: 947 of 1020

 Retrying in 1 seconds

Evaluating input: 948 of 1020

 Retrying in 1 seconds

Evaluating input: 949 of 1020

 Retrying in 1 seconds

Evaluating input: 950 of 1020

 Retrying in 1 seconds

Evaluating input: 951 of 1020

 Retrying in 1 seconds

Evaluating input: 952 of 1020

 Retrying in 1 seconds

Evaluating input: 953 of 1020

 Retrying in 1 seconds

Evaluating input: 954 of 1020

 Retrying in 1 seconds

Evaluating input: 955 of 1020

 Retrying in 1 seconds

Evaluating input: 956 of 1020

 Retrying in 1 seconds

Evaluating input: 957 of 1020

 Retrying in 1 seconds

Evaluating input: 958 of 1020

 Retrying in 1 seconds

Evaluatin

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [8]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = 98.22275158075254
Optimization restart 2/5, f = 98.22275166684187
Optimization restart 3/5, f = 115.56822075498896
Optimization restart 4/5, f = 98.22275158295989
Optimization restart 5/5, f = 98.22275158250889


GP_regression.,value,constraints,priors
rbf.variance,4484.460876294901,+ve,
rbf.lengthscale,18.650523183457544,+ve,
Gaussian_noise.variance,269.9007071554226,+ve,


#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)


#### Experimental Design Loop

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [12]:
ed_loop.run_loop(user_function_co2, 1000)

Optimization restart 1/5, f = 98.22275158075254
Optimization restart 2/5, f = 115.56809998030971


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 3/5, f = 98.22275158076937
Optimization restart 4/5, f = 115.56819595719125
Optimization restart 5/5, f = 115.56808735374737
[[ 3.  8. 27. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 103.06960083552761
Optimization restart 2/5, f = 120.98966783198317
Optimization restart 3/5, f = 103.06567650039592
Optimization restart 4/5, f = 120.98965868724312
Optimization restart 5/5, f = 120.9896765923182
[[20.  8. 27. 30.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


...

Optimization restart 1/5, f = 4245.251279767908
Optimization restart 2/5, f = 4245.250950887192
Optimization restart 3/5, f = 4245.250950887369
Optimization restart 4/5, f = 4245.2509508870635


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 4245.250950942928
[[ 3.         25.          5.         49.73608081  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 4248.8177512529155
Optimization restart 2/5, f = 4248.8176624588905
Optimization restart 3/5, f = 4248.817662470874
Optimization restart 4/5, f = 4248.817662458753
Optimization restart 5/5, f = 4375.41414094292


### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [13]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = 98.22275158075254
Optimization restart 2/5, f = 115.56808446782004
Optimization restart 3/5, f = 115.56819515938179
Optimization restart 4/5, f = 98.22275162616326
Optimization restart 5/5, f = 98.22275158039436


#### Acquisition Function

In [14]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ird, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [17]:
ed_loop.run_loop(user_function_co2, 1000)

Optimization restart 1/5, f = 98.22275158039436
Optimization restart 2/5, f = 115.56810933010411
Optimization restart 3/5, f = 98.22275158197024
Optimization restart 4/5, f = 103.88407417148967
Optimization restart 5/5, f = 98.2227515826133
[[ 8.         10.27847276 21.44052963 68.19856265  2.          4.23414227]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 102.88379134958393
Optimization restart 2/5, f = 102.88360622950803
Optimization restart 3/5, f = 108.50942525877112
Optimization restart 4/5, f = 102.88360622999676
Optimization restart 5/5, f = 121.13233462605267
[[13.          8.91099603 26.59181602 30.46974422  2.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 1/5, f = 107.4337280621701
Optimization restart 2/5, f = 126.7242969451062
Optimization restart 3/5, f = 107.4337280407089
Optimization restart 4/5, f = 107.43372805827474
Optimization restart 5/5, f = 126.72428614408422
[[ 3.         21.56725979 27.         39.95054482  2.          4.34817244]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


...

Optimization restart 1/5, f = 4237.962679327761
Optimization restart 2/5, f = 4237.962295760701
Optimization restart 3/5, f = 4237.962295756919
Optimization restart 4/5, f = 4237.9622962494595
Optimization restart 5/5, f = 4237.962295756106
[[11.         19.56052351  6.02078089 39.82723083  3.          4.63140352]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 4241.378005914721
Optimization restart 2/5, f = 4241.377631691543
Optimization restart 3/5, f = 4241.377631452893
Optimization restart 4/5, f = 4241.37763143741
Optimization restart 5/5, f = 4241.377632537956
[[16.         15.00173214 13.22398913 34.43840976  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 4245.087216063618
Optimization restart 2/5, f = 4245.086977919547
Optimization restart 3/5, f = 4245.086978038313
Optimization restart 4/5, f = 4245.086977922632
Optimization restart 5/5, f = 4245.086978304424


### Random GP

Supervised learning

In [18]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=5)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/5, f = 4283.341676221597
Optimization restart 2/5, f = 4283.341676995652
Optimization restart 3/5, f = 4283.341676245787
Optimization restart 4/5, f = 4283.34167622159
Optimization restart 5/5, f = 4283.341676222196


GP_regression.,value,constraints,priors
rbf.variance,36307.46815406588,+ve,
rbf.lengthscale,18.53107617467358,+ve,
Gaussian_noise.variance,182.309749860952,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [19]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  13.852128421412335


In [20]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  13.815853079421954


In [21]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  13.674554909849586


In [22]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is -1.299% better than training on random points.'

In [23]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is -1.033% better than training on random points.'

### Save Models

In [24]:
with open(f'experimental_design/models/model_variance_20_init_points_1000_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design/models/integrated_variance_reduction_20_init_points_1000_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design/models/1020_random_points_supervised_GP_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)

In [25]:
with open(f'experimental_design/models/co2-1000-pts.pkl', 'wb') as f:
    pickle.dump((train_X, train_Y), f)